In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('./funda_train.csv')
train.head()

FileNotFoundError: [Errno 2] File b'./funda_train.csv' does not exist: b'./funda_train.csv'

In [ ]:
train[train['amount']<0].head()

In [ ]:
train = train.drop(['region','type_of_business'],axis=1)
train.head()

In [ ]:
# 거래일와 거래시간을 합친 변수를 생성합니다.
train['datetime'] = pd.to_datetime(train.transacted_date + " " + 
                                train.transacted_time, format='%Y-%m-%d %H:%M:%S')
## 환불 거래를 제거하는 함수를 정의합니다.
def remove_refund(df):
    refund = df[df['amount']<0] # 매출액 음숫값 데이터를 추출합니다.
    non_refund = df[df['amount']>0] # 매출액 양숫값 데이터를 추출합니다.
    removed_data = pd.DataFrame()
    
    for i in tqdm(df.store_id.unique()):
        # 매출액이 양숫값인 데이터를 상점별로 나눕니다.
        divided_data = non_refund[non_refund['store_id']==i] 
        # 매출액이 음숫값인 데이터를 상점별로 나눕니다.
        divided_data2 = refund[refund['store_id']==i] 
        
        for neg in divided_data2.to_records()[:]: # 환불 데이터를 차례대로 검사합니다.
            refund_store = neg['store_id']
            refund_id = neg['card_id'] # 환불 카드 아이디를 추출합니다.
            refund_datetime = neg['datetime'] # 환불 시간을 추출합니다.
            refund_amount = abs(neg['amount']) # 매출 음숫값의 절댓값을 구합니다.
                
            ## 환불 시간 이전의 데이터 중 카드 이이디와 환불액이 같은 후보 리스트를 뽑습니다.
            refund_pay_list = divided_data[divided_data['datetime']<=refund_datetime]
            refund_pay_list = refund_pay_list[refund_pay_list['card_id']==refund_id]
            refund_pay_list = refund_pay_list[refund_pay_list['amount']==refund_amount]
                     
            # 후보 리스트가 있으면 카드 아이디, 환불액이 같으면서 가장 최근시간을 제거합니다.
            if(len(refund_pay_list)!=0):
                refund_datetime = max(refund_pay_list['datetime']) # 가장 최근 시간을 구합니다
                noise_list = divided_data[divided_data['datetime']==refund_datetime] # 가장 최근시간
                noise_list = noise_list[noise_list['card_id']==refund_id] # 환불 카드 아이디
                noise_list = noise_list[noise_list['amount']==refund_amount] ## 환불액
                divided_data = divided_data.drop(index=noise_list.index) # 인덱스를 통해 제거
                    
        ## 제거한 데이터를 데이터프레임에 추가합니다.
        removed_data = pd.concat([removed_data,divided_data],axis=0)
    
    return removed_data